In [1]:
import torch
from torch.utils.cpp_extension import load

if torch.cuda.is_available():
    device = torch.device("cuda")

In [2]:
ultMul = load(name='ultMul', sources=['ultMul_cuda_new.cpp', 'ultMul_cuda_kernel_new.cu'])

import ultmul as um
um.ultmul

<function ultMul.PyCapsule.ultMul>

In [3]:
import sklearn.metrics.pairwise as sk_dist
torch.manual_seed(11000)
test = torch.randn(10001,10).to(device).float() # number of samples have to smaller than sqrt(512) need to fix
euc_dist = torch.from_numpy(sk_dist.euclidean_distances(test.to('cpu').numpy())).to(device).float()
print(euc_dist.size())
euc_dist

torch.Size([10001, 10001])


tensor([[0.0000, 3.8651, 3.7960,  ..., 5.0696, 4.0650, 3.8771],
        [3.8651, 0.0000, 3.6884,  ..., 3.6742, 3.8784, 3.2865],
        [3.7960, 3.6884, 0.0000,  ..., 5.0290, 3.1813, 2.8613],
        ...,
        [5.0696, 3.6742, 5.0290,  ..., 0.0000, 4.4713, 5.3355],
        [4.0650, 3.8784, 3.1813,  ..., 4.4713, 0.0000, 3.6948],
        [3.8771, 3.2865, 2.8613,  ..., 5.3355, 3.6948, 0.0000]],
       device='cuda:0')

In [4]:
print(euc_dist[0,:])
print(euc_dist[1,:])

tensor([0.0000, 3.8651, 3.7960,  ..., 5.0696, 4.0650, 3.8771], device='cuda:0')
tensor([3.8651, 0.0000, 3.6884,  ..., 3.6742, 3.8784, 3.2865], device='cuda:0')


In [5]:
#euc_dist
um.ultmul(euc_dist, euc_dist)

tensor([[0.0000, 2.3477, 2.2658,  ..., 2.7678, 2.4993, 2.4654],
        [2.3477, 0.0000, 2.4644,  ..., 2.3378, 2.2386, 1.8229],
        [2.2658, 2.4644, 0.0000,  ..., 2.9297, 2.0087, 2.1461],
        ...,
        [2.7678, 2.3378, 2.9297,  ..., 0.0000, 2.6144, 3.0921],
        [2.4993, 2.2386, 2.0087,  ..., 2.6144, 0.0000, 2.1844],
        [2.4654, 1.8229, 2.1461,  ..., 3.0921, 2.1844, 0.0000]],
       device='cuda:0')

In [6]:
um.clusterability(euc_dist)

[tensor([43]), tensor([[0.0000, 1.5714, 1.5605,  ..., 1.6542, 1.4745, 1.5650],
         [1.5714, 0.0000, 1.5714,  ..., 1.6542, 1.5714, 1.5714],
         [1.5605, 1.5714, 0.0000,  ..., 1.6542, 1.5605, 1.5650],
         ...,
         [1.6542, 1.6542, 1.6542,  ..., 0.0000, 1.6542, 1.6542],
         [1.4745, 1.5714, 1.5605,  ..., 1.6542, 0.0000, 1.5650],
         [1.5650, 1.5714, 1.5650,  ..., 1.6542, 1.5650, 0.0000]],
        device='cuda:0')]

In [8]:
rlt_dist_o = euc_dist.clone()#torch.empty(euc_dist.size())
rlt_dist_n = um.ultmul(rlt_dist_o, euc_dist)
i = 1
#print(rlt_dist_n)
#print(rlt_dist_o)
while not torch.eq(rlt_dist_o, rlt_dist_n).all().item():
    rlt_dist_o = rlt_dist_n #result of previous multiplication
    rlt_dist_n = um.ultmul(rlt_dist_o, euc_dist) # new result
    i = i+1
print(i)
print(rlt_dist_n)

um.ultmul(rlt_dist_n, euc_dist)

43
tensor([[0.0000, 1.5714, 1.5605,  ..., 1.6542, 1.4745, 1.5650],
        [1.5714, 0.0000, 1.5714,  ..., 1.6542, 1.5714, 1.5714],
        [1.5605, 1.5714, 0.0000,  ..., 1.6542, 1.5605, 1.5650],
        ...,
        [1.6542, 1.6542, 1.6542,  ..., 0.0000, 1.6542, 1.6542],
        [1.4745, 1.5714, 1.5605,  ..., 1.6542, 0.0000, 1.5650],
        [1.5650, 1.5714, 1.5650,  ..., 1.6542, 1.5650, 0.0000]],
       device='cuda:0')


tensor([[0.0000, 1.5714, 1.5605,  ..., 1.6542, 1.4745, 1.5650],
        [1.5714, 0.0000, 1.5714,  ..., 1.6542, 1.5714, 1.5714],
        [1.5605, 1.5714, 0.0000,  ..., 1.6542, 1.5605, 1.5650],
        ...,
        [1.6542, 1.6542, 1.6542,  ..., 0.0000, 1.6542, 1.6542],
        [1.4745, 1.5714, 1.5605,  ..., 1.6542, 0.0000, 1.5650],
        [1.5650, 1.5714, 1.5650,  ..., 1.6542, 1.5650, 0.0000]],
       device='cuda:0')

In [ ]:
torch.matmul(euc_dist, euc_dist)